In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse

In [2]:
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 200)

In [ ]:
#HOW our pages look like
#https://bilim.akipress.org/profiles:3/school/list//page:2
#https://bilim.akipress.org/profiles:3/school/list//page:23

In [12]:
#GET the url with a function
def pages(page_number):
    url = 'https://bilim.akipress.org/profiles:3/school/list//page:' + str(page_number)
    return url

In [62]:
#GET all the urls with a loop and SAVE to a list
urls=[]
for x in range(2, 24):
    urls.append(pages(x))
urls

['https://bilim.akipress.org/profiles:3/school/list//page:2',
 'https://bilim.akipress.org/profiles:3/school/list//page:3',
 'https://bilim.akipress.org/profiles:3/school/list//page:4',
 'https://bilim.akipress.org/profiles:3/school/list//page:5',
 'https://bilim.akipress.org/profiles:3/school/list//page:6',
 'https://bilim.akipress.org/profiles:3/school/list//page:7',
 'https://bilim.akipress.org/profiles:3/school/list//page:8',
 'https://bilim.akipress.org/profiles:3/school/list//page:9',
 'https://bilim.akipress.org/profiles:3/school/list//page:10',
 'https://bilim.akipress.org/profiles:3/school/list//page:11',
 'https://bilim.akipress.org/profiles:3/school/list//page:12',
 'https://bilim.akipress.org/profiles:3/school/list//page:13',
 'https://bilim.akipress.org/profiles:3/school/list//page:14',
 'https://bilim.akipress.org/profiles:3/school/list//page:15',
 'https://bilim.akipress.org/profiles:3/school/list//page:16',
 'https://bilim.akipress.org/profiles:3/school/list//page:17',


In [63]:
schools=[]
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.select('table', class_='table-striped')
    table = table[0]
    #schools_list = []
    schools_trs = soup.select('tr')
    for tr in schools_trs[1:]:
        tds = tr.select('td')
        td_list = []
        for (i, td) in enumerate(tds):
            if(i >= 1):
                td_list.append(td.text)
            #if it's the second column, get the href link also
            if(i == 1):
                td_list.append(td.find('a')['href']) 
            #schools_list.append(td_list)
        schools.append(td_list)

In [64]:
df = pd.DataFrame(schools, columns=["name", "link", "address"])
df['urls'] = ("https://bilim.akipress.org"+df['link'])
df.shape

(2122, 4)

In [65]:
df['contents'] = df.urls.apply(lambda url: requests.get(url).text)

In [66]:
def get_score(content):
    soup = BeautifulSoup(content)
    tables = soup.select("table")
    try:
        trs = tables[2].find_all('tr')
        for (i, tr) in enumerate (trs):
           #if it's the second row, go there
            if (i == 1):
                tds = tr.find_all('td')
                for (i, td) in enumerate(tds):
           # if it's the third column, get the value
                    if(i == 2):
                        score = td.text
        return score
    except:
        pass

In [67]:
score = df.contents.apply(get_score)
score = score.to_frame().reset_index()
score.columns = ['index', 'value']

In [69]:
df['score'] = score['value']

In [71]:
df1=df

In [72]:
del df1['contents']

In [73]:
df1.head()

,name,link,address,urls,score
0,СШ им.Эшенкулова,/profile:1099/ssh-imeshenkulova/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Тору-Айгыр 722302 ул.ентибаева 5",https://bilim.akipress.org/profile:1099/ssh-imeshenkulova/?from=bilim&place=list,108.8
1,СШ им.Иманкулова,/profile:1085/ssh-imimankulova/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Бактуу-Долоноту Гагарина № 5 722317",https://bilim.akipress.org/profile:1085/ssh-imimankulova/?from=bilim&place=list,95.3
2,СШ им.Бейшекеева,/profile:1074/ssh-imbeishekeeva/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Кароол-Дебе ул.Шадыканова б\н722327",https://bilim.akipress.org/profile:1074/ssh-imbeishekeeva/?from=bilim&place=list,126
3,СШ им.Орозова,/profile:1084/ssh-imorozova/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Корумду Сокубай № 37 722318",https://bilim.akipress.org/profile:1084/ssh-imorozova/?from=bilim&place=list,109.1
4,СШ им.Абдрахманова,/profile:1077/ssh-imabdrahmanova/?from=bilim&place=list,"Ыссык-Кульская область, Ыссык-Кульский район c.Ананьево ул.Ленина №65 722324",https://bilim.akipress.org/profile:1077/ssh-imabdrahmanova/?from=bilim&place=list,121.6


In [74]:
df1.to_csv('scores1.csv', index = False)